<a href="https://colab.research.google.com/github/sliscak/notebooks/blob/main/Stable_Diffusion%2BDIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Generates an image from text using [Stable Diffusion](https://github.com/CompVis/stable-diffusion) from [Diffusers](https://github.com/huggingface/diffusers) library and makes a mask with [DIS](https://github.com/xuebinqin/DIS.git) that is used to seperate the image into foreground and background with PIL.

---




### Install requirements

In [1]:
%%capture
!pip install --upgrade diffusers
!pip install --upgrade gradio
!pip install --upgrade transformers
!pip install --upgrade ftfy
!git clone https://github.com/xuebinqin/DIS

In [2]:
import gradio as gr
import os
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline
from google.colab import output
from huggingface_hub import notebook_login
from transformers import DPTFeatureExtractor, DPTForDepthEstimation
import torch
import numpy as np
from PIL import Image
import requests

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [3]:
output.enable_custom_widget_manager()

In [4]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-c6aba92b-caf6-0bec-c6ad-212e6b88e74b)


In [5]:
# login to verify license
notebook_login()

In [6]:
!pwd
# import sys
# sys.path.insert(0,'/content/DIS/IS-Net')
%cd DIS
!pwd

/content
/content/DIS
/content/DIS


In [7]:
!git lfs install
%cd saved_models
!git clone https://huggingface.co/NimaBoscarino/IS-Net_DIS-general-use

Updated git hooks.
Git LFS initialized.
/content/DIS/saved_models
Cloning into 'IS-Net_DIS-general-use'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 22 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [8]:
!mv IS-Net_DIS-general-use IS-Net
%cd ..
!rm -rf demo_datasets
!mkdir demo_datasets
%cd demo_datasets
!mkdir your_dataset
!mkdir your_dataset_result
%cd ..

/content/DIS
/content/DIS/demo_datasets
/content/DIS


In [9]:
%cd IS-Net
!python Inference.py

/content/DIS/IS-Net
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
0it [00:00, ?it/s]


In [10]:
# execfile('Inference.py')

In [11]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cuda'
# pipe = StableDiffusionPipeline.from_pretrained("hakurei/waifu-diffusion", use_auth_token=False)
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", use_auth_token=True)
pipe = pipe.to(device)

Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/342 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/743 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/522 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/335M [00:00<?, ?B/s]

In [ ]:
torch.cuda.empty_cache()
def get_image(prompt):
    # print(prompt)
    with autocast(device):
        image = pipe(prompt).images[0].resize((640, 640))
        image.save('../demo_datasets/your_dataset/image.png')
    return image, '../demo_datasets/your_dataset/image.png'

def get_mask(image):
    # image.save('../demo_datasets/your_dataset/image.png')
    # execfile('Inference.py')
    exec(open('Inference.py').read())
    image = Image.open('../demo_datasets/your_dataset_result/image.png')
    return image, '../demo_datasets/your_dataset_result/image.png'

def get_mesh(prompt):
    # print(prompt)
    with autocast(device):
        image = pipe(prompt).images[0]
    return image

def separate(image,mask):
    mask = mask.convert('L')
    black_image = Image.new(mode="RGB", size=(640, 640))
    foreground = Image.composite(image, black_image, mask)
    background = Image.composite(black_image, image, mask)

    foreground.save('../demo_datasets/your_dataset_result/foreground.png')
    background.save('../demo_datasets/your_dataset_result/background.png')
    return foreground, background, '../demo_datasets/your_dataset_result/foreground.png','../demo_datasets/your_dataset_result/background.png'
# def remove_background():


black_image = Image.new(mode="RGB", size=(640, 640))
with gr.Blocks() as demo:
    with gr.Row():
        text = gr.Text()
        image = gr.Image(interactive=True, label='output image', value=black_image, type='pil')
        # image = gr.Image(interactive=True, label='output image', tool='sketch', value=black_image)
        mask = gr.Image(interactive=False, label='mask', type='pil')
        foreground = gr.Image(interactive=True, label='foreground', value=black_image, type='pil')
        background = gr.Image(interactive=True, label='background', value=black_image, type='pil')
        # mesh = gr.Model3D(clear_color=[0.0, 0.0, 0.0, 0.0],  label="3D Model")
        # with gr.Column():
    with gr.Row():
        button = gr.Button('generate image')
    with gr.Row():
        button2 = gr.Button('generate mask')
    with gr.Row():
        button3 = gr.Button('separate foreground from background')
    with gr.Row():
        image_file = gr.File(interactive=False, file_count=1, label='image')
        mask_file = gr.File(interactive=False, file_count=1, label='mask')
        foreground_file = gr.File(interactive=False, file_count=1, label='foreground')
        background_file = gr.File(interactive=False, file_count=1, label='background')
    button.click(fn=get_image, inputs=text, outputs=[image, image_file])
    button2.click(fn=get_mask, inputs=image, outputs=[mask, mask_file])
    button3.click(fn=separate, inputs=[image, mask], outputs=[foreground, background, foreground_file, background_file])

demo.launch(debug=False)